## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.2 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.7.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 52s]
val_accuracy: 0.906166672706604

Best val_accuracy So Far: 0.9085000157356262
Total elapsed time: 00h 04m 53s
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner_search.results_summary()

Results summary
Results in output/Mnist Fashion
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 3
dense_1_units: 64
learning_rate: 0.001
Score: 0.9085000157356262
Trial summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 5
conv_2_filter: 64
conv_2_kernel: 5
dense_1_units: 48
learning_rate: 0.001
Score: 0.906166672706604
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 3
dense_1_units: 112
learning_rate: 0.01
Score: 0.875333309173584
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 80
learning_rate: 0.01
Score: 0.8634999990463257
Trial summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 5
conv_2_filter: 64
conv_2_kernel: 3
dense_1_units: 112
learning_rate: 0.01
Score: 0.8538333177566528


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        13872     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 64)                1486912   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,502,266
Trainable params: 1,502,266
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 15s 8ms/step - loss: 0.1648 - accuracy: 0.9383 - val_loss: 0.2425 - val_accuracy: 0.9157
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1264 - accuracy: 0.9527 - val_loss: 0.2590 - val_accuracy: 0.9137
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0981 - accuracy: 0.9635 - val_loss: 0.2809 - val_accuracy: 0.9090
Epoch 7/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0717 - accuracy: 0.9733 - val_loss: 0.3600 - val_accuracy: 0.9097
Epoch 8/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0569 - accuracy: 0.9790 - val_loss: 0.3364 - val_accuracy: 0.9113
Epoch 9/10
1688/1688 [==============================] - 14s 9ms/step - loss: 0.0447 - accuracy: 0.9835 - val_loss: 0.4011 - val_accuracy: 0.9143
Epoch 10/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0353 - accuracy: 0.9874 - val_loss: 0.4462 - val_a